### Surface Analysis (Examples)

This notebook provides visualizations of surface analysis produced derived from a combination of smartphone and MADIS observations.

1. Import relevant Python libraries and setup cartopy/colortables

2. Retrieve composite reflectivity analysis, smartphone pressure analysis, and MADIS (Kalman smoothed) temperature, dew point, and wind analyses for 14 April, 2018.  

3. Generate animations of temperature, moisture, and pressure analsyes for the 14 April, 2018. 

4. Demonstrate how mesoscale temperature, moisture, and wind perturbations are extracted from analyses using band-pass filtering. Plot and save mesoscale pressure perturbations and composite reflectivity analyses over the two-day period from 14-15 May, 2018. From the saved images, produce a movies showing mesoscale pressure perturbations associated with convection.

In [1]:
### ---- (1) ---- ####
#Import Python libraries

import os
import xarray as xr
import matplotlib
import cmasher as cmr
from matplotlib import pyplot as plt
import numpy as np
import funcs
import sys
import lcmaps
import colorcet as cc
import cmasher as cmr
from datetime import datetime,timedelta
from cartopy.feature import NaturalEarthFeature,BORDERS,LAKES,COLORS
import cartopy.crs as crs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from metpy.plots import colortables
from scipy import ndimage
from scipy import signal
from scipy.signal import butter, lfilter
import multiprocessing
from joblib import Parallel,delayed

#Retrieve perceptually uniform colorbar from colorcet
cmapp = cc.cm.rainbow_bgyrm_35_85_c71

#Set format for datetime objects
fmt = '%Y%m%d_%H%M'

# Download/add state and coastline features for cartopy 
states = NaturalEarthFeature(category="cultural", scale="10m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")

land_50m = NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k',
                                        facecolor='none')

#Define function to add map data to matplotlib plot
def add_map(ax,clr,lw):
    ax.add_feature(states)
    ax.add_feature(BORDERS)
    ax.add_feature(land_50m)
    ax.add_feature(states,edgecolor=clr,lw=lw)
    ax.add_feature(LAKES, edgecolor=clr)

#Define function to add latitude/longitude grid lines to cartopy/matplotlib plot
def add_gridlines(ax,xl,yl,clr, fs):
    gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                      linewidth=0.25, color=clr, alpha=1, linestyle='--')

    gl.xlabels_bottom = xl
    gl.xlabels_top = False
    gl.ylabels_left = yl
    gl.ylabels_right = False

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': fs, 'color': clr}
    gl.ylabel_style = {'size': fs, 'color': clr}
    return gl

#Get Composite Reflectivity colormap from metpy
ctable1 = 'NWSStormClearReflectivity'
cmapp = cc.cm.rainbow_bgyrm_35_85_c71
norm, cmapp_radar = colortables.get_with_steps(ctable1, 244, 244)

#Increase with of notebook to fill screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#Define function to mask pressure analyses over water
def mask_grid(arr):
    arr = np.ma.masked_where(landsea==0,arr)
    return arr

#Define function to read and subset a land/sea boolean grid
def get_landsea():
    ds_land = xr.open_dataset('../data/Static/landsea.nc')
    ds_land = funcs.subset(ds_land,minLat,maxLat,minLng,maxLng)
    landsea = ds_land['LANDSEA'].values
    landsea = np.pad(landsea, ((0,1),(0,1)), 'edge')
    ds_land.close()
    return landsea

In [2]:
#---- (2) ---- #

#Set date, date format, and observation type
day = '20180414'
fmt = '%Y%m%d_%H%M'

#Define bounding box for analysis
minLng = -105.5; maxLng = -70.5; minLat = 28.5; maxLat = 48.5
#Get land/sea boolean within bounding box
landsea = get_landsea()

#Retrieve pre-generated Kalman smoothed LatticeKrig pressure analysis for 14 April, 2018.
ds_all = xr.open_dataset('../data/KF/kfsmart_full_altimeter_'+day+'.nc')
#Convert analysis time to list of datetime objects
dts = ds_all['Valid'].values 
dtlist = [datetime.utcfromtimestamp(d/1e9).strftime(fmt) for d in dts.tolist()]

#Retrieve temperature grid and latitude/longitude dims
alts_kf = ds_all['altimeter_rts'].values
ygrid = ds_all['longitude'].values; xgrid = ds_all['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Generate 2D coordinate grid for contouring
ds_all.close()

#Retrieve pre-generated Kalman smoothed LatticeKrig temperature analysis for 14 April, 2018.
ds_all2 = xr.open_dataset('../data/KF/kfmadis_full_temperature_'+day+'.nc')
#Retrieve temperature grid and latitude/longitude dims
temp_kf = (9/5.0)*(ds_all2['temperature_rts'].values-273.15) + 32
ds_all2.close()

#Retrieve pre-generated Kalman smoothed LatticeKrig dew point analysis for 14 April, 2018.
ds_all3 = xr.open_dataset('../data/KF/kfmadis_full_dewpoint_'+day+'.nc')
#Retrieve temperature grid and latitude/longitude dims
dwpt_kf = (9/5.0)*(ds_all3['dewpoint_rts'].values-273.15) + 32
ds_all3.close()

#Retrieve pre-generated Kalman smoothed LatticeKrig wind analysis for 14 April, 2018.
ds_all4 = xr.open_dataset('../data/KF/kfmadis_full_wind_'+day+'.nc')
#Retrieve temperature grid and latitude/longitude dims
uwind_kf = ds_all4['uwind_rts'].values*1.94384
vwind_kf = ds_all4['vwind_rts'].values*1.94384
ds_all4.close()

#Retrieve pre-generated Kalman smoothed LatticeKrig wind analysis for 14 April, 2018.
ds_all5 = xr.open_dataset('../data/Radar/cref_'+day+'.nc')
refl = ds_all5['radar_composite'].values
ds_all5.close()

In [7]:
#---- (3) ---- #

#Get lower and upper bounds for colorbars
def get_mm(mmeso,split=False):
    mn = round(np.nanmin(mmeso),0)
    mx = round(np.nanmax(mmeso),0)
    if (split):
            if (abs(mn) > mx):
                    mx = -1*mn
    return mn,mx

#Get min,max limits for colormaps
amin,amax = get_mm(alts_kf)
tmin,tmax = get_mm(temp_kf)
tdmin,tdmax = get_mm(temp_kf)

cmapp_temp = lcmaps.thetae() #Retrieve temp colormap

#Define function to plot pressure anlyses and save to a png file in the Plots directory
def plot_tmp(d,ddate):    
    
    #Mask temperature analysis over water
    temp_kf_2d = mask_grid(temp_kf[d])
    uwind_kf_2d = mask_grid(uwind_kf[d])
    vwind_kf_2d = mask_grid(vwind_kf[d])

    #Smooth temperature analysis for contouring (use higher smoothing level for synoptic analysis)
    temp_kf_2d_smooth = ndimage.gaussian_filter(temp_kf_2d,sigma=1)
    
    #Initiatlize figure
    fig =plt.figure(figsize=(18,10))
    ax1 = plt.subplot(111,projection=crs.PlateCarree()) #Set projection
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',20) #Add grid lines and x/y labels  
    
    im = plt.pcolormesh(X,Y,temp_kf_2d,cmap=cmapp_temp,vmin=tmin,vmax=tmax)
    cs1 = plt.contour(X,Y,mask_grid(temp_kf_2d_smooth),levels=np.arange(tmin,32,4),linestyles='--',colors='k')
    cs2 = plt.contour(X,Y,mask_grid(temp_kf_2d_smooth),levels=np.arange(32,tmax,4),colors='k')

    ddx = 15
    # Plot wind barbs
    ax1.barbs(X[::ddx,::ddx],Y[::ddx,::ddx],uwind_kf_2d[::ddx,::ddx],vwind_kf_2d[::ddx,::ddx],length=6)#,regrid_shape=20)

    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('2-m Temperature ($\degree$F), Wind Barbs (kt): '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($^\circ$F)',y=1.02,fontsize=20) #Set colorbar title
    cb.ax.tick_params(labelsize=20) #Set colorbar tick size
    fig.canvas.draw()
    plt.tight_layout()
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    plt.savefig('../data/Plots/'+day+'/kftemp_'+dd+'.png')
    plt.close()

#Define function to plot pressure anlyses and save to a png file in the Plots directory
def plot_dpt(d,ddate):    
    #Mask temperature analysis over water
    dwpt_kf_2d = mask_grid(dwpt_kf[d])
    uwind_kf_2d = mask_grid(uwind_kf[d])
    vwind_kf_2d = mask_grid(vwind_kf[d])

    #Smooth temperature analysis for contouring (use higher smoothing level for synoptic analysis)
    dwpt_kf_2d_smooth = ndimage.gaussian_filter(dwpt_kf_2d,sigma=1)
    
    #Initiatlize figure
    fig =plt.figure(figsize=(18,10))
    ax1 = plt.subplot(111,projection=crs.PlateCarree()) #Set projection
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',20) #Add grid lines and x/y labels  
    
    im = plt.pcolormesh(X,Y,dwpt_kf_2d,cmap=cmapp_temp,vmin=tdmin,vmax=tdmax)
    cs1 = plt.contour(X,Y,mask_grid(dwpt_kf_2d_smooth),levels=np.arange(tdmin,32,4),linestyles='--',colors='k')
    cs2 = plt.contour(X,Y,mask_grid(dwpt_kf_2d_smooth),levels=np.arange(32,tdmax,4),colors='k')

    ddx = 15
    # Plot wind barbs
    ax1.barbs(X[::ddx,::ddx],Y[::ddx,::ddx],uwind_kf_2d[::ddx,::ddx],vwind_kf_2d[::ddx,::ddx],length=6)#,regrid_shape=20)

    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('2-m Dew Point ($\degree$F), Wind Barbs (kt): '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($^\circ$F)',y=1.02,fontsize=20) #Set colorbar title
    cb.ax.tick_params(labelsize=20) #Set colorbar tick size
    fig.canvas.draw()
    plt.tight_layout()
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    plt.savefig('../data/Plots/'+day+'/kfdwpt_'+dd+'.png')
    plt.close()

#Define function to plot pressure anlyses and save to a png file in the Plots directory
def plot_rfl(d,ddate):    
    #Mask temperature analysis over water
    refl_2d = mask_grid(refl[d])
    alts_kf_2d = mask_grid(alts_kf[d])
    uwind_kf_2d = mask_grid(uwind_kf[d])
    vwind_kf_2d = mask_grid(vwind_kf[d])

    #Smooth temperature analysis for contouring (use higher smoothing level for synoptic analysis)
    alts_kf_2d_smooth = ndimage.gaussian_filter(alts_kf_2d,sigma=3)
    
    #Initiatlize figure
    fig =plt.figure(figsize=(18,10))
    ax1 = plt.subplot(111,projection=crs.PlateCarree()) #Set projection
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',20) #Add grid lines and x/y labels  
    
    im = plt.pcolormesh(X,Y,refl_2d,cmap=cmapp_radar,vmin=-32,vmax=90)
    #Contour pressure every 2 hPa
    CS = ax1.contour(X,Y,mask_grid(alts_kf_2d_smooth),levels=np.arange(amin,amax+2,2),colors='k',alpha=1,lw=2)
    ax1.clabel(CS, CS.levels, inline=True, fmt="%1.f", fontsize=16, colors='k') #Add contour labels
    
    ddx = 15
    # Plot wind barbs
    ax1.barbs(X[::ddx,::ddx],Y[::ddx,::ddx],uwind_kf_2d[::ddx,::ddx],vwind_kf_2d[::ddx,::ddx],length=6,color='dimgray')#,regrid_shape=20)

    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('Altimeter ($hPa$), Wind Barbs ($kt$), Composite Reflectivity (dBZ): '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($dBZ$)',y=1.02,fontsize=20) #Set colorbar title
    cb.ax.tick_params(labelsize=20) #Set colorbar tick size
    fig.canvas.draw()
    plt.tight_layout()
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    plt.savefig('../data/Plots/'+day+'/kfrfl_'+dd+'.png')
    plt.close()

In [9]:
#Perform temperature plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_tmp)(d,ddate) for d,ddate in enumerate(dtlist))

In [10]:
#Perform dew point plotting in parallel (one plot - per core)
results = Parallel(n_jobs=num_cores)(delayed(plot_dpt)(d,ddate) for d,ddate in enumerate(dtlist))

In [11]:
#Perform altimeter (reflectivity) plotting in parallel (one plot - per core)
results = Parallel(n_jobs=num_cores)(delayed(plot_rfl)(d,ddate) for d,ddate in enumerate(dtlist))

In [12]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day+'/kftemp_'+day+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day+'/kftemp_'+day+'.mp4')
    
#Create mp4 movie from 5-min pressure anlayses saved as pngs
os.system('ffmpeg -r 9 -f image2 -s 1920x1080 -i ../data/Plots/'+day+'/kftemp_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day+'/kftemp_'+day+'.mp4')
#(Below) Display MADIS pressure analysis for 14 April, 2018

0

In [13]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src = "../data/Plots/20180414/kftemp_20180414.mp4" type="video/mp4">
</video></div>

In [14]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day+'/kfdwpt_'+day+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day+'/kfdwpt_'+day+'.mp4')
    
#Create mp4 movie from 5-min pressure anlayses saved as pngs
os.system('ffmpeg -r 9 -f image2 -s 1920x1080 -i ../data/Plots/'+day+'/kfdwpt_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day+'/kfdwpt_'+day+'.mp4')
#(Below) Display MADIS dew point analysis for 14 April, 2018

0

In [15]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src = "../data/Plots/20180414/kfdwpt_20180414.mp4" type="video/mp4">
</video></div>

In [16]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day+'/kfrfl_'+day+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day+'/kfrfl_'+day+'.mp4')
    
#Create mp4 movie from 5-min pressure anlayses saved as pngs
os.system('ffmpeg -r 9 -f image2 -s 1920x1080 -i ../data/Plots/'+day+'/kfrfl_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day+'/kfrfl_'+day+'.mp4')
#(Below) Display MADIS dew point analysis for 14 April, 2018

0

In [17]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src = "../data/Plots/20180414/kfrfl_20180414.mp4" type="video/mp4">
</video></div>

In [20]:
#---- (4) ---- #

#Define dates of analysis and observation type
day1 = '20180514'
day2 = '20180515'
otyp = 'temperature'

#Define bounding box
minLng = -83.0; maxLng = -70.5; minLat = 38.5; maxLat= 45.0

#Get land/sea boolean within bounding box
landsea = get_landsea()
landsea = landsea[:-1,:]

#Retrieve temperature analyses for each day 14-15 of May, 2018
ds1 = xr.open_dataset('../data/KF/kfmadis_full_'+otyp+'_'+day1+'.nc')
ds2 = xr.open_dataset('../data/KF/kfmadis_full_'+otyp+'_'+day2+'.nc')

#Combine pressure analysis from each day into a single xarray dataset
ds_all = xr.concat([ds1,ds2],'Valid')

#Convert list of analysis times to list of datetime objects
dts = ds_all['Valid'].values
dtlist = [datetime.utcfromtimestamp(d/1e9).strftime(fmt) for d in dts.tolist()]

#Get temperature and reflectivity analyses over the two-day period (14-15, May 2018) 
temp_kf = ds_all[otyp+'_rts'].values

#Get dimensions of temperature anlaysis
ys = temp_kf.shape[1] #lat dim
xs = temp_kf.shape[2] #lng dim

#Set upper and lower limits of band pass filter
highcut = 1/(3600*2)
lowcut = 1/(3600*6)
fs = 1/300.0 #Set temporal frequency of analysis (in hertz)
order=2

#Define highpass/bandpass filter function
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs #nyquist frequency
        low = lowcut / nyq #lower limit filter
        high = highcut / nyq #upper limit of filter
        b, a = butter(order, [low, high], btype='band') #Perform bandpass
        #b, a = butter(order, [high], btype = 'highpass')
        return b, a

#Function to perform band pass filtering
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = signal.filtfilt(b, a, data)
        return y

#Filter time-series
def filter_ts(vvar):
        meso = butter_bandpass_filter(vvar, lowcut, highcut, fs, order)
        return meso

#Function to perform bandpass filtering of pressure time series at grid point (y,x)
def perform_filter(vvar,x,y):
    if (y == xs-1):
            print(x)
    
    #Get bandpass filtered time-series
    meso = filter_ts(vvar)
    return meso

#Get list of latitude/longitude pairs for every grid point in the analysis domain
xy_pair = [];
for i in range(0,ys):
    for j in range(0,xs):
        xy_pair.append((i,j))
        
        
def run_bpass(otyp,ntyp):

    #Retrieve pressure analyses for each day 14-15 of May, 2018
    if ('wind' in otyp):
        ds1 = xr.open_dataset('../data/KF/kf'+ntyp+'_full_wind_'+day1+'.nc')
        ds2 = xr.open_dataset('../data/KF/kf'+ntyp+'_full_wind_'+day2+'.nc')
    else:
        ds1 = xr.open_dataset('../data/KF/kf'+ntyp+'_full_'+otyp+'_'+day1+'.nc')
        ds2 = xr.open_dataset('../data/KF/kf'+ntyp+'_full_'+otyp+'_'+day2+'.nc')
        
    #Combine pressure analysis from each day into a single xarray dataset
    ds_all = xr.concat([ds1,ds2],'Valid')
    ds1.close()
    ds2.close()
    
    vvar_kf = ds_all[otyp+'_rts'].values
    
    #Perform bandpass filtering in parallel
    num_cores = multiprocessing.cpu_count()
    vvar_meso = Parallel(n_jobs=num_cores)(delayed(perform_filter)(vvar_kf[:,x,y],x,y) for x,y in xy_pair)

    #Reshape the bandpass filtered pressure dataset so its dimensions are (Time, Latitude, Longitude)
    vvar_meso = np.float32(vvar_meso)
    nshp = (ys,xs,len(dtlist))
    vvar_meso = np.reshape(vvar_meso,nshp).T
    vvar_meso = np.swapaxes(vvar_meso,1,2)

    #Write bandpass filtered pressure perturbations to NetCDF
    ds = xr.Dataset()
    ds[otyp+'_meso'] = xr.DataArray(vvar_meso,coords={'Valid':dts,'latitude':ds_all['latitude'].values,'longitude':ds_all['longitude'].values},dims=('Valid','latitude','longitude'))
    ds.to_netcdf('../data/KF/kf'+ntyp+'_bpass_'+otyp+'_'+day2+'.nc')

In [21]:
#Band pass filter (2-6h) smartphone altimeter analysis
run_bpass('altimeter','smart')

In [22]:
#Band pass filter (2-6h) madis temperature analysis
run_bpass('temperature','madis')

In [23]:
#Band pass filter (2-6h) madis dewpoint analysis
run_bpass('dewpoint','madis')

In [24]:
#Band pass filter (2-6h) madis uwind analysis
run_bpass('uwind','madis')

In [25]:
#Band pass filter (2-6h) madis temperature analysis
run_bpass('vwind','madis')

In [46]:
#Set base font size
matplotlib.rcParams.update({'font.size': 20})

dg1 = xr.open_dataset('../data/KF/kfsmart_bpass_altimeter_'+day2+'.nc')
alts_meso = dg1['altimeter_meso'].values
dg1.close()

dg2 = xr.open_dataset('../data/KF/kfmadis_bpass_temperature_'+day2+'.nc')
temp_meso = dg2['temperature_meso'].values
dg2.close()

dg3 = xr.open_dataset('../data/KF/kfmadis_bpass_dewpoint_'+day2+'.nc')
dewp_meso = dg3['dewpoint_meso'].values
dg3.close()

dg4 = xr.open_dataset('../data/KF/kfmadis_bpass_uwind_'+day2+'.nc')
uwind_meso = dg4['uwind_meso'].values
dg4.close()

dg5 = xr.open_dataset('../data/KF/kfmadis_bpass_vwind_'+day2+'.nc')
vwind_meso = dg5['vwind_meso'].values
dg5.close()

#Retrieve pre-generated Kalman smoothed LatticeKrig wind analysis for 14 April, 2018.
dg6 = xr.open_dataset('../data/Radar/cref_201805.nc')
refl2 = dg6['REFL'].values
dg6.close()

#Retrieve latitude/longitude from dataset
ygrid = dg5['longitude'].values; xgrid = dg5['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Create 2d coordinates for contour plotting

alt_min = np.round(np.nanmin(alts_meso)-0.5,0)
alt_max = np.round(np.nanmax(alts_meso)+0.5,0)
if (abs(alt_min)>alt_max):
        alt_max = -1*alt_min
else:
        alt_min = -1*alt_max

aran = [-1.25,-1,-0.75,0.75,1,1.25] #list(np.arange(alt_min,alt_max,0.75))

tmp_min = np.round(np.nanmin(temp_meso)-1,0)
tmp_max = np.round(np.nanmax(temp_meso)+1,0)
if (abs(tmp_min)>tmp_max):
        tmp_max = -1*tmp_min
else:
        tmp_min = -1*tmp_max

tran = list(np.arange(tmp_min,tmp_max,0.5))
tran.remove(0)

dpt_min = np.round(np.nanmin(dewp_meso)-1,0)
dpt_max = np.round(np.nanmax(dewp_meso)+1,0)
if (abs(dpt_min)>dpt_max):
        dpt_max = -1*dpt_min
else:
        dpt_min = -1*dpt_max

dran = list(np.arange(dpt_min,dpt_max,0.5))
dran.remove(0)


matplotlib.rcParams.update({'font.size': 16})

def plot_pert(d,ddate):
    
    d=d+144 #Start at 1200 UTC 14 May 
      
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(221,projection=crs.PlateCarree())
    add_map(ax1,'dimgray',1)
    add_gridlines(ax1,False,True,'k',18)

    im1 = ax1.imshow(alts_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.PuOr_r,vmin=-1.5,vmax=1.5)
    im11 = ax1.contour(X,Y,alts_meso[d],levels=aran,colors='k')
    q1 = ax1.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)
    plt.colorbar(im1)
    ax1.set_title("Altimeter (hPa)",fontsize=16)

    ax2 = plt.subplot(222,projection=crs.PlateCarree())
    add_map(ax2,'dimgray',1)
    add_gridlines(ax2,False,False,'k',18)

    im2 = ax2.imshow(temp_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.RdBu_r,vmin=-1.5,vmax=1.5)
    im22 = ax2.contour(X,Y,temp_meso[d],levels=tran,colors='k')
    q2 = ax2.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax2.set_title("Temperature (deg C)",fontsize=16)
    plt.colorbar(im2)

    ax3 = plt.subplot(223,projection=crs.PlateCarree())
    add_map(ax3,'dimgray',1)
    add_gridlines(ax3,True,True,'k',18)

    im3 = ax3.imshow(dewp_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.BrBG,vmin=-1.5,vmax=1.5)
    im33 = ax3.contour(X,Y,dewp_meso[d],levels=dran,colors='k')
    q3 = ax3.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax3.set_title("Dew Point (deg C)",fontsize=16)
    plt.colorbar(im3)

    ax4 = plt.subplot(224,projection=crs.PlateCarree())
    add_map(ax4,'dimgray',1)
    add_gridlines(ax4,True,False,'k',18)

    im4 = ax4.imshow(refl2[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp_radar,vmin=-32,vmax=90)
    cs11 = ax4.contour(X,Y,alts_meso[d],levels=aran,colors='k')
    q4 = ax4.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax4.clabel(cs11,cs11.levels,fmt="%1.2f",inline=True,fontsize=9)
    ax4.set_title("CREF (dBz)",fontsize=16)
    plt.colorbar(im4)

    #Save image with %03d format for animation with ffmpeg
    d = d-144
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
        
    plt.suptitle('5-min Perturbation Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    fig.canvas.draw()
    plt.tight_layout()
    plt.savefig('../data/Plots/'+day2+'/kfpert_surface_'+dd+'.png')
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_pert)(d,ddate) for d,ddate in enumerate(dtlist[144:]))

In [47]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day2+'/kfpert_surface_'+day2+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day2+'/kfpert_surface_'+day2+'.mp4')
#Create mp4 movie from 5-min pressure perturbation / reflectivity anlayses saved as pngs
os.system('ffmpeg -r 12 -f image2 -s 1920x1080 -i ../data/Plots/'+day2+'/kfpert_surface_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day2+'/kfpert_surface_'+day2+'.mp4')
#(Below) display video

0

In [48]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../data/Plots/20180515/kfpert_surface_20180515.mp4" type="video/mp4">
</video></div>